In [112]:
import pandas as pd
import seaborn as sns
sns.set_theme(style="white")

from valentine.algorithms import Coma
from valentine.algorithms import JaccardDistanceMatcher
from valentine.algorithms import SimilarityFlooding
from valentine.algorithms import DistributionBased
from valentine.algorithms import Cupid
from valentine.metrics import F1Score, PrecisionTopNPercent
from valentine import valentine_match
import pprint

import os
import sys
import jellyfish

sys.path.append(os.path.join(os.path.abspath(''), '..'))
from gdc.gdc_api import GDCSchema

pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)


In [113]:
df_target = pd.read_csv('../data/target.csv')
df_target.drop('study', axis=1, inplace=True)
print(df_target.shape)
df_target.head(5)

(1068, 16)


,case_submitter_id,age_at_diagnosis,race,ethnicity,gender,vital_status,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_stage,tumor_grade,tumor_focality,tumor_largest_dimension_diameter,primary_diagnosis,morphology,tissue_or_organ_of_origin,tumor_code
0,01BR001,20089.0,black or african american,not hispanic or latino,female,Alive,T2,N1c,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA
1,01BR008,17532.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
2,01BR009,23376.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
3,01BR010,23741.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
4,01BR015,12784.0,white,not hispanic or latino,female,Alive,T2,N1,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA


In [114]:
df_liu = pd.read_csv('../data/liuData_Supplementary_Table_1.csv')
df_liu_dictionary = pd.read_csv('../data/liuData_Data_dictionary.csv')
print(df_liu.shape)
df_liu.head(5)

(1185, 83)


,Case_ID,tumor_code,tumor_sample_id_protein,normal_sample_id_protein,is_excluded_from_pancancer_studies,reason_for_exclusion,is_gtex,specimen/aliquout_id_protein_normal,specimen/aliquout_id_protein_tumor,specimen/aliquout_id_RNA_Tumor,...,follow-up/additional_treatment_radiation_therapy_for_new_tumor,follow-up/additional_treatment_pharmaceutical_therapy_for_new_tumor,follow-up/additional_treatment_immuno_for_new_tumor,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_loco-regional,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_metastasis,"Recurrence-free survival, days","Recurrence status (1, yes; 0, no)","Overall survival, days","Survival status (1, dead; 0, alive)","Time between collection and diagnosis, days"
0,11BR047,BRCA,11BR047-T,NaN,NaN,NaN,0.0,NaN,81116212-b7e6-454b-9579-105cf3,ef52c640-13a9-4855-9ce2-0be77a_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,369.0,0.0,NaN
1,11BR043,BRCA,11BR043-T,NaN,NaN,NaN,0.0,NaN,6d34d499-167e-42aa-9790-316fca_D2,6d34d499-167e-42aa-9790-316fca_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,374.0,0.0,NaN
2,11BR049,BRCA,11BR049-T,NaN,NaN,NaN,0.0,NaN,2e700669-85b0-43fa-a9c7-3eaf5a_D2,2e700669-85b0-43fa-a9c7-3eaf5a_D1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,379.0,0.0,NaN
3,11BR023,BRCA,11BR023-T,NaN,NaN,NaN,0.0,NaN,0a80d3c4-0758-447a-958c-ea868c,079b5600-6afc-4785-bb22-48cfab_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,398.0,0.0,NaN
4,18BR010,BRCA,18BR010-T,NaN,NaN,NaN,0.0,NaN,0bb9d596-774e-452b-9c89-a6643c_D2,0bb9d596-774e-452b-9c89-a6643c_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,410.0,0.0,NaN


In [115]:
threshold = len(df_liu) * 0.5
df_liu.dropna(thresh=threshold, axis=1, inplace=True)
print(df_liu.shape)
df_liu.head(10)


(1185, 51)


,Case_ID,tumor_code,tumor_sample_id_protein,normal_sample_id_protein,is_gtex,specimen/aliquout_id_protein_normal,specimen/aliquout_id_protein_tumor,specimen/aliquout_id_RNA_Tumor,specimen/aliquout_id_DNA_Tumor,specimen/aliquout_id_DNA_Blood,...,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_last_contact,follow-up/adjuvant_post-operative_radiation_therapy,follow-up/adjuvant_post-operative_pharmaceutical_therapy,follow-up/adjuvant_post-operative_immunological_therapy,follow-up/measure_of_success_of_outcome_at_the_completion_of_initial_first_course_treatment,follow-up/new_tumor_after_initial_treatment,"Recurrence status (1, yes; 0, no)","Overall survival, days","Survival status (1, dead; 0, alive)","Time between collection and diagnosis, days"
0,11BR047,BRCA,11BR047-T,NaN,0.0,NaN,81116212-b7e6-454b-9579-105cf3,ef52c640-13a9-4855-9ce2-0be77a_D7_1,ef52c640-13a9-4855-9ce2-0be77a_D6_1,0bfd3ae8-d275-4dab-9360-d1df30_D1_1,...,369.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,369.0,0.0,NaN
1,11BR043,BRCA,11BR043-T,NaN,0.0,NaN,6d34d499-167e-42aa-9790-316fca_D2,6d34d499-167e-42aa-9790-316fca_D7_1,6d34d499-167e-42aa-9790-316fca_D6_1,a1338e3d-598a-45b5-838f-9b6206_D1_1,...,374.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,374.0,0.0,NaN
2,11BR049,BRCA,11BR049-T,NaN,0.0,NaN,2e700669-85b0-43fa-a9c7-3eaf5a_D2,2e700669-85b0-43fa-a9c7-3eaf5a_D1,2e700669-85b0-43fa-a9c7-3eaf5a_D6,ad85fadf-31b7-44f3-b6ea-73c55d_D1,...,379.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,379.0,0.0,NaN
3,11BR023,BRCA,11BR023-T,NaN,0.0,NaN,0a80d3c4-0758-447a-958c-ea868c,079b5600-6afc-4785-bb22-48cfab_D7_1,079b5600-6afc-4785-bb22-48cfab_D6_1,70ee4720-f4dc-4b91-a6aa-c0efd9_D1_1,...,398.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,398.0,0.0,NaN
4,18BR010,BRCA,18BR010-T,NaN,0.0,NaN,0bb9d596-774e-452b-9c89-a6643c_D2,0bb9d596-774e-452b-9c89-a6643c_D7_1,0bb9d596-774e-452b-9c89-a6643c_D6_1,feab8720-9694-4679-917f-f1287c_D1_1,...,410.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,410.0,0.0,NaN
5,06BR003,BRCA,06BR003-T,NaN,0.0,NaN,31b13596-554e-452f-91a4-7e67a8_D2,"['31b13596-554e-452f-91a4-7e67a8_D7', 'not run']","['31b13596-554e-452f-91a4-7e67a8_D6', 'not run']","['31da4db6-8404-43f0-995d-ebc10b_D1', 'not run']",...,323.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,323.0,0.0,NaN
6,11BR074,BRCA,11BR074-T,NaN,0.0,NaN,3367406e-d39c-4641-a3e7-44e1f3,12ce8313-6c54-4bee-a996-1aa7f8_D7,12ce8313-6c54-4bee-a996-1aa7f8_D6_1,93f961e1-fe4c-4dfc-8cbe-1dff31_D1_1,...,367.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,367.0,0.0,NaN
7,18BR017,BRCA,18BR017-T,NaN,0.0,NaN,392b8aa4-9f99-4250-b693-326260_D2,392b8aa4-9f99-4250-b693-326260_D7_1,392b8aa4-9f99-4250-b693-326260_D6_1,f49705bd-129b-4bfe-a944-a7eb51_D1_1,...,445.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,445.0,0.0,NaN
8,01BR017,BRCA,01BR017-T,NaN,0.0,NaN,09659708-7747-4d59-a3b9-e221e0_D2,09659708-7747-4d59-a3b9-e221e0_D7,09659708-7747-4d59-a3b9-e221e0_D6,b76c68fc-2403-4e79-ad46-4ae336_D1,...,413.0,Yes,Yes,NaN,NaN,No,0.0,413.0,0.0,NaN
9,06BR006,BRCA,06BR006-T,NaN,0.0,NaN,3799eeb5-c966-4059-a78f-ace269_D2,3799eeb5-c966-4059-a78f-ace269_D7,3799eeb5-c966-4059-a78f-ace269_D6,7e168c2a-f22a-44d9-ac7d-1a4c42_D1,...,137.0,No,Yes,NaN,NaN,Yes,1.0,137.0,0.0,NaN


## Compute matching between GDC_compiled file and Liu_data

In [116]:
matcher = Coma()
# matcher = Coma(use_instances=True)
# matcher = JaccardDistanceMatcher()
# matcher = SimilarityFlooding()
# matcher = DistributionBased()
# matcher = Cupid()

print(f"Computing the matches using the {type(matcher).__name__} algorithm...")

matches = valentine_match(df_liu, df_target, matcher)

Computing the matches using the Coma algorithm...


In [117]:
print("Found the following matches:")
pp.pprint(matches)



Found the following matches:
{   (('table_1', 'Case_ID'), ('table_2', 'case_submitter_id')): 0.50803775,
    (('table_1', 'Ethnicity'), ('table_2', 'ethnicity')): 0.7851225,
    (('table_1', 'Race'), ('table_2', 'race')): 0.7710247,
    (('table_1', 'baseline/margin_status'), ('table_2', 'vital_status')): 0.37917706,
    (('table_1', 'baseline/tumor_focality'), ('table_2', 'tumor_focality')): 0.5914982,
    (('table_1', 'baseline/tumor_stage_pathological'), ('table_2', 'ajcc_pathologic_stage')): 0.42711246,
    (('table_1', 'cptac_path/histologic_grade'), ('table_2', 'ajcc_pathologic_t')): 0.38114378,
    (('table_1', 'tumor_code'), ('table_2', 'tumor_code')): 0.78709614}


In [118]:
print("\nOne-to-one matches:")
pp.pprint(matches.one_to_one())


One-to-one matches:
{   (('table_1', 'Case_ID'), ('table_2', 'case_submitter_id')): 0.50803775,
    (('table_1', 'Ethnicity'), ('table_2', 'ethnicity')): 0.7851225,
    (('table_1', 'Race'), ('table_2', 'race')): 0.7710247,
    (('table_1', 'baseline/tumor_focality'), ('table_2', 'tumor_focality')): 0.5914982,
    (('table_1', 'tumor_code'), ('table_2', 'tumor_code')): 0.78709614}


In [119]:
target_cols = df_target.columns.str.lower()
liu_cols = df_liu.columns.str.lower()

for candidate in liu_cols:
    if candidate in target_cols:
        print(f"Column {candidate} is present in both datasets")
    else:
        for target in target_cols:
            if jellyfish.jaro_similarity(candidate, target) > 0.7:
                print(f"Column {candidate} matches with {target}")
        

Column case_id matches with case_submitter_id
Column tumor_code is present in both datasets
Column tumor_sample_id_protein matches with tumor_grade
Column tumor_sample_id_protein matches with tumor_focality
Column tumor_sample_id_protein matches with tumor_largest_dimension_diameter
Column age matches with age_at_diagnosis
Column age matches with race
Column race is present in both datasets
Column ethnicity is present in both datasets
Column baseline/tumor_focality matches with tumor_focality
Column cptac_path/histologic_grade matches with ajcc_pathologic_stage


### Generate a table from GDC dictionary and check for matching


In [131]:
for col in df_target.columns:
    probe = 'clinical::'+col
    print(probe)
    gdc_schema = GDCSchema(probe)
    entry = gdc_schema.get_properties_by_gdc_candidate(probe)
    if isinstance(entry, dict):
        domain = gdc_schema.get_gdc_col_values()
        print(domain)
    else:
        pass
    
    
    
   

clinical::case_submitter_id
clinical::age_at_diagnosis
(-inf, inf)
clinical::race
['not reported', 'white', 'american indian or alaska native', 'black or african american', 'asian', 'native hawaiian or other pacific islander', 'other']
clinical::ethnicity
['hispanic or latino', 'not hispanic or latino']
clinical::gender
['female', 'male', 'unknown', 'unspecified']
clinical::vital_status
['alive', 'dead', 'lost to follow-up']
clinical::ajcc_pathologic_t
clinical::ajcc_pathologic_n
clinical::ajcc_pathologic_stage
clinical::tumor_grade
clinical::tumor_focality
clinical::tumor_largest_dimension_diameter
clinical::primary_diagnosis
clinical::morphology
clinical::tissue_or_organ_of_origin
clinical::tumor_code


In [ ]:
gdc_schema = GDCSchema(ajcc_pathologic_stage)
gdc_schema.candidates.keys()